# Configuration

In [ ]:
CONNECTION_STRING = "mongodb://localhost:27017"
DATABASE_NAME = "off"
COLLECTION_NAME = "products"

# Nutritional values based on nutritional grade, healthy ingredients, and additives used.

In [ ]:
from pymongo import MongoClient
from bson.code import Code
import plotly
plotly.offline.init_notebook_mode()
from plotly.graph_objs import Bar

client = MongoClient(CONNECTION_STRING)
db = client[DATABASE_NAME]
openfood = db[COLLECTION_NAME]

In order to calculate nutritional values, we need to search for relevant fields that can be used to statistically calculate nutritional value.

In [ ]:
mapper = Code("""
    function () {
        for (var key in this) {
            emit(key, null);
        }
    }""")
reducer = Code("""
    function (key, values) {
        return null;
    }""")

openfood_fields = openfood.map_reduce(mapper, reducer, "openfood_fields")
for each in openfood_fields.find():
    print(each['_id'])

In [ ]:
mapper = Code("""
    function () {
        if (typeof this.nutriments !== 'undefined' && this.nutriments !== "") {
            for (var key in this.nutriments) {
                emit(key, 1);
            }
        }
    }""")
reducer = Code("""
    function (key, values) {
        var total = 0;
        for (var i = 0; i < values.length; i++) {
            total += values[i];
        }
        return total;
    }""")

openfood_fields = openfood.map_reduce(mapper, reducer, "nutriments_fields")
count = 0
for each in openfood_fields.find():
    count += 1
    print(each['_id'], each['value'])
    
print(count)

Each food entry states the countries which the food it is sold. Below, we try to find out the list of countries which the food are sold.

In [ ]:
# mapper = Code("""
#     function () {
#         if (typeof this.countries_tags !== 'undefined') {
#             emit(this.countries_tags[0], 1);
#         } else {
#             emit("None", 1);
#         }
        
#     }""")
mapper = Code("""
    function () {
        if (typeof this.countries !== 'undefined' && this.countries !== "") {
            var tmp_split = this.countries.split(",");
            for (i = 0; i < tmp_split.length; i++) {
                emit(tmp_split[i], 1);
            }
            
        } else {
            emit("None", 1);
        }
        
    }""")
reducer = Code("""
    function (key, values) {
        var total = 0;
        for (var i = 0; i < values.length; i++) {
            total += values[i];
        }
        return total;
    }""")

countries_stats = openfood.map_reduce(mapper, reducer, "results")
for each in countries_stats.find():
    print(each['_id'], each['value'])

In [ ]:
country = []
count = []
for each in countries_stats.find():
    country.append(each['_id'])
    count.append(each['value'])
data = [Bar(x=country, y=count)]
plotly.offline.iplot(data, filename='country-stat-bar')

In [ ]:
help(Bar)